In [ ]:
!pip install --upgrade pip
!pip install unsloth
!pip install datasets  # for loading MRPC
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 20.9 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 MB 80.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.5/31.5 MB 76.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.2/865.2 MB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 152.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 181.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 52.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 35.5 MB/s eta 

In [ ]:
from datasets import load_dataset

# Load MRPC (train + validation + test splits)
mrpc_data = load_dataset("glue", "mrpc")
train_data = mrpc_data["train"]
val_data   = mrpc_data["validation"]


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/649k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/75.7k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/308k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [ ]:
train_data[0]

{'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .',
 'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .',
 'label': 1,
 'idx': 0}

In [ ]:
def format_example(ex):
    label_text = "Yes" if ex["label"] == 1 else "No"
    text = (
        "Determine whether the following two sentences are paraphrases of each other. Respond with 'Yes' if they are paraphrases, and 'No' otherwise.\n"
        f"Sentence1: {ex['sentence1']}\n"
        f"Sentence2: {ex['sentence2']}\n"
        f"Paraphrase: {label_text}."
    )
    return {"text": text}

train_data = train_data.map(format_example)
val_data   = val_data.map(format_example)


Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

In [ ]:
train_data[0]

{'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .',
 'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .',
 'label': 1,
 'idx': 0,
 'text': 'Determine whether the following two sentences are paraphrases of each other. Respond with \'Yes\' if they are paraphrases, and \'No\' otherwise.\nSentence1: Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .\nSentence2: Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .\nParaphrase: Yes.'}

In [ ]:
from unsloth import FastLanguageModel

model_Llama1B, tokenizer_Llama1B= FastLanguageModel.from_pretrained(
  model_name= "unsloth/Llama-3.2-1B",
  max_seq_length=2048,
  dtype=None,
  load_in_4bit=False  # or True/False depending on memory; 4-bit will use less memory
)

model_Llama1Binstruct, tokenizer_Llama1Binstruct= FastLanguageModel.from_pretrained(
  model_name= "unsloth/Llama-3.2-1B-Instruct",
  max_seq_length=2048,
  dtype=None,
  load_in_4bit=False
)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.4.7: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.7k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

chat_template.jinja:   0%|          | 0.00/3.83k [00:00<?, ?B/s]

In [ ]:
model_Llama1B = FastLanguageModel.get_peft_model(
    model_Llama1B,
    r=8,
    target_modules=["q_proj","k_proj","v_proj"],
    lora_alpha=16,
    lora_dropout=0
)

model_Llama1Binstruct = FastLanguageModel.get_peft_model(
    model_Llama1Binstruct,
    r=8,
    target_modules=["q_proj","k_proj","v_proj"],
    lora_alpha=16,
    lora_dropout=0
)


Not an error, but Unsloth cannot patch MLP layers with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Not an error, but Unsloth cannot patch O projection layer with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Unsloth 2025.4.7 patched 16 layers with 16 QKV layers, 0 O layers and 0 MLP layers.


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq

training_args = TrainingArguments(
    output_dir="llama3.2:1b",
    per_device_train_batch_size=4,   # adjust based on GPU memory
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=1,   # to simulate larger batch if needed
    logging_steps=20,
    weight_decay=0.01,
    learning_rate=3e-4,
    num_train_epochs=20,

)

trainer = SFTTrainer(
    model=model_Llama1B,
    tokenizer=tokenizer_Llama1B,
    train_dataset=train_data,
    eval_dataset=val_data,
    dataset_text_field="text",
    data_collator=DataCollatorForSeq2Seq(tokenizer_Llama1B),
    args=training_args,
)

training_args_instruct = TrainingArguments(
    output_dir="llama3.2:1b_instruct",
    per_device_train_batch_size=4,   # adjust based on GPU memory
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=1,   # to simulate larger batch if needed
    logging_steps=10,
    weight_decay=0.01,
    learning_rate=3e-4,
    num_train_epochs=20,

)

trainer_instruct = SFTTrainer(
    model=model_Llama1Binstruct,
    tokenizer=tokenizer_Llama1Binstruct,
    train_dataset=train_data,
    eval_dataset=val_data,
    dataset_text_field="text",
    data_collator=DataCollatorForSeq2Seq(tokenizer_Llama1Binstruct),
    args=training_args_instruct,
)


Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/3668 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/408 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/3668 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/408 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 3,668 | Num Epochs = 20 | Total steps = 18,340
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 1 x 1) = 4
 "-____-"     Trainable parameters = 1,179,648/1,236,994,048 (0.10% trained)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: mohamed-hendy (mohamed-hendy-new-york-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
20,1.774400
40,1.292400
60,1.240300
80,1.273500
100,1.193400
120,1.237300
140,1.209800
160,1.192200
180,1.176200
200,1.238300


Unsloth: Will smartly offload gradients to save VRAM!


TrainOutput(global_step=18340, training_loss=0.6107638982026111, metrics={'train_runtime': 4018.735, 'train_samples_per_second': 18.255, 'train_steps_per_second': 4.564, 'total_flos': 4.65594035785728e+16, 'train_loss': 0.6107638982026111})

In [ ]:
torch.save(model_Llama1Binstruct.state_dict(), '/content/drive/MyDrive/model_Llama1B.pth')

In [ ]:
trainer_instruct.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 3,668 | Num Epochs = 20 | Total steps = 18,340
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 1 x 1) = 4
 "-____-"     Trainable parameters = 1,179,648/1,236,994,048 (0.10% trained)


Step,Training Loss
10,2.288600
20,1.637000
30,1.440800
40,1.408100
50,1.391600
60,1.329200
70,1.399500
80,1.342000
90,1.309900
100,1.264500


/usr/local/lib/python3.11/dist-packages/peft/utils/other.py:1110: UserWarning: Unable to fetch remote file due to the following error (ProtocolError('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')), '(Request ID: dc130c0e-b2b8-41d1-a7c5-d49c445a9545)') - silently ignoring the lookup for the file config.json in unsloth/Llama-3.2-1B-Instruct.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:236: UserWarning: Could not find a config file in unsloth/Llama-3.2-1B-Instruct - will assume that the vocabulary was not modified.
  warnings.warn(


In [ ]:

torch.save(model_Llama1Binstruct.state_dict(), '/content/drive/MyDrive/model_Llama1Binstruct.pth')


In [ ]:
import torch
import evaluate

model_Llama1B.eval()
predictions, references = [], []
i = 0
for ex in val_data:

    prompt = ex["text"].rsplit("Paraphrase:", 1)[0] + "Paraphrase:"  # remove label
    inputs = tokenizer_Llama1B(prompt, return_tensors="pt").to(model_Llama1B.device)
    # print("the example is:", ex)
    with torch.no_grad():
        outputs = model_Llama1B.generate(**inputs, max_new_tokens=5)

    decoded = tokenizer_Llama1B.decode(outputs[0], skip_special_tokens=True)
    pred_token = decoded.lower().split("paraphrase:")[-1].strip().split()[0]

    predictions.append(1 if pred_token.startswith("yes") else 0)
    # print("the pred_token is:", pred_token)
    references.append(ex["label"])

metric = evaluate.load("glue", "mrpc")
result = metric.compute(predictions=predictions, references=references)
print("the results for the normal llama are:\n", result)


the results for the normal llama are:
 {'accuracy': 0.7867647058823529, 'f1': 0.8471001757469244}


In [ ]:
import torch

model_Llama1Binstruct.eval()
predictions, references = [], []
i = 0
for ex in val_data:

    prompt = ex["text"].rsplit("Paraphrase:", 1)[0] + "Paraphrase:"  # remove label
    inputs = tokenizer_Llama1Binstruct(prompt, return_tensors="pt").to(model_Llama1B.device)
    # print("the example is:", ex)
    with torch.no_grad():
        outputs = model_Llama1Binstruct.generate(**inputs, max_new_tokens=5)

    decoded = tokenizer_Llama1Binstruct.decode(outputs[0], skip_special_tokens=True)
    pred_token = decoded.lower().split("paraphrase:")[-1].strip().split()[0]

    predictions.append(1 if pred_token.startswith("yes") else 0)
    # print("the pred_token is:", pred_token)
    references.append(ex["label"])

result = metric.compute(predictions=predictions, references=references)
print("the results for the instruct llama are:\n", result)

the results for the instruct llama are:
 {'accuracy': 0.7892156862745098, 'f1': 0.8458781362007168}


In [ ]:
## test on the test set for the final scores

{'accuracy': 0.803921568627451, 'f1': 0.8571428571428571}


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
